<a href="https://colab.research.google.com/github/evmi7/motyli-ai-vyhledavac/blob/main/GradioClipApp-Motyli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🪴 Gradio aplikace: Vyhledávání podobných motýlů pomocí CLIP
# Vyhledávání podle textu nebo obrázku

import gradio as gr
import clip
import torch
from PIL import Image
import numpy as np
import pickle
import os
from datasets import load_dataset

# 📅 Pokus o načtení embeddingů
embedding_path = "butterfly_embeddings.pt"
if os.path.exists(embedding_path):
    with open(embedding_path, "rb") as f:
        data = pickle.load(f)
        image_embeddings = data["embeddings"]
        image_labels = data["tags"]
        image_ids = data["paths"]
        label_names = data.get("label_names", None)
    embeddings_loaded = True
else:
    embeddings_loaded = False

# 🧠 Načtení CLIP modelu
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 📆 Načtení datasetu
hf_dataset = load_dataset("imagefolder", repo_id="evmi7/motyli-butterflays-full2", split="train")

# ⚖️ Funkce vyhledávání podle textu

def search_by_text(query, top_k=5):
    if not embeddings_loaded:
        return None, [None] * top_k

    text = clip.tokenize([query]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    similarities = (image_embeddings @ text_features.cpu().numpy().T).squeeze()
    best_indices = similarities.argsort()[-top_k:][::-1]
    images = [hf_dataset[int(image_ids[i])]["image"] for i in best_indices]
    captions = [label_names[image_labels[i]] if label_names else str(image_labels[i]) for i in best_indices]
    return None, list(zip(images, captions))

# 🔍 Funkce vyhledávání podle obrázku

def search_by_image(image, top_k=5):
    if not embeddings_loaded:
        return image, [None] * top_k

    image_input = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    similarities = (image_embeddings @ image_features.cpu().numpy().T).squeeze()
    best_indices = similarities.argsort()[-top_k:][::-1]
    images = [hf_dataset[int(image_ids[i])]["image"] for i in best_indices]
    captions = [label_names[image_labels[i]] if label_names else str(image_labels[i]) for i in best_indices]
    return image, list(zip(images, captions))

# 📊 Rozhraní aplikace
with gr.Blocks() as demo:
    gr.Markdown("# 🪴 Vyhledávání motýlů pomocí CLIP")
    gr.Markdown("Zadej anglický textový dotaz nebo nahraj obrázek. Aplikace vrátí nejpodobnější motýly.")

    with gr.Row():
        text_input = gr.Textbox(label="Textový dotaz", placeholder="e.g. orange butterfly with black spots")
        image_input = gr.Image(type="pil", label="Nebo nahraj referenční obrázek")

    search_button = gr.Button("Vyhledat")
    query_image = gr.Image(label="📸 Referenční obrázek", visible=False)
    gallery = gr.Gallery(label="🎭 Nejpodobnější motýli", columns=5, rows=1)

    def run_search(text, image):
        if text:
            return search_by_text(text)
        elif image:
            return search_by_image(image)
        else:
            return None, []

    search_button.click(fn=run_search, inputs=[text_input, image_input], outputs=[query_image, gallery])

    if not embeddings_loaded:
        gr.Markdown("### ⚠️ Embeddingy zatím nejsou dostupné. Jakmile budou nahrány, aplikace začne vracet výsledky.")

demo.launch()
